In [41]:
import numpy as np
import pyvisa 
import time
import csv
import matplotlib.pyplot as plt

from IPython.display import display, clear_output

rm = pyvisa.ResourceManager()

k2400_heater = rm.open_resource('GPIB0::2::INSTR')
k2400_sample_current = rm.open_resource('GPIB0::1::INSTR') #good
k2000_pt = rm.open_resource('GPIB0::26::INSTR') #good
k2000_sample_voltage = rm.open_resource('GPIB0::3::INSTR') #good

def get_current(device):
    return float(device.query(':MEASure:curr?').split(',')[1])

def get_voltage(device):
    return float(device.query(':MEASure:voltage?'))

def get_fres(device):
    return float(device.query(':MEASure:FRES?'))

def set_current(device, value):
    device.write(f':sour:current {value}')
    device.write(':outp on')

def set_current_range(device, value):
    device.write(f':sour:curr:range {value}')

def get_current_range(device):
    return device.query(':sour:curr:rang?')

def temp_fres(weerstand):
    #lineair:
    #source: https://www.sterlingsensors.co.uk/pt100-resistance-table
    #weerstand (ohm) -- #temp (K)
    #100 -- 273.15 -->(x2,y2)
    #18.49 -- 73.15 --> (x1, y1)
    (x1, y1) = (18.49, 73.15)
    (x2, y2) = (31.32, 103.15)
    temp = y1 + (y2-y1)/(x2-x1) * (weerstand - x1)
    return temp

def get_temp(device):
    fpres = get_fres(device)
    temp = temp_fres(fpres)
    return temp

def I_V_scan(I_vals, write_location = None):
    max_I = max(I_vals)
    if max_I == 0:
        raise ValueError
    else:
        i = 0
        while 10**(-i) >= max_I:
            i +=1
        range_max = 10**(-i+1)
    set_current_range(k2400_sample_current, range_max)
    I_actuals = []
    V_vals = []
    Temps = []
    for I in I_vals:
        set_current(k2400_sample_current, I)
        time.sleep(0.5)
        get_current(k2400_sample_current)
        I_actuals.append(get_current(k2400_sample_current))
        V_vals.append(get_voltage(k2000_sample_voltage))
        Temps.append(get_temp(k2000_pt))
    
    if write_location is not None:
        file = open(write_location, "at+")
        file.write("I = ")
        file.write(str(I_actuals ))
        file.write('\n') 
        file.write("V = "+str(V_vals) + '\n')
        file.write('T = '+str(Temps) + '\n')
        file.close()
    return [I_actuals, V_vals, Temps]


def R_T_scan(write_location, current):
    Temps = []
    Resistances = []
    highT = True
    set_current(k2400_sample_current, current)
    try:
        while True:
            temp = get_temp(k2000_pt)
            if temp < 100:
                highT = False
            volt = get_voltage(k2000_sample_voltage)
            amp = get_current(k2400_sample_current)
            print('measured')
            Temps.append(temp)
            Resistances.append(volt/amp)
            if highT:
                time.sleep(10)
            else:
                time.sleep(1)
    except KeyboardInterrupt:
        file = open(write_location, "at+")
        file.write('T = ' + str(Temps) + '\n')
        file.write('R = ' + str(Resistances) + '\n')
        



In [43]:
R_T_scan('Data/R_T_testing3.txt', .1)

measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured


In [38]:
k2400_sample_current.write(':outp off')

11

In [39]:
set_current(k2400_sample_current, .5)

In [13]:
k2000_pt.query(':MEASure:FRES?')

'+2.19640358E+01\n'

In [20]:
V = I_V_scan([0.01,0.02,0.03,0.04,0.05,0.06], "Data/test1.txt")

In [22]:
namen = ["Data/test2.txt", "Data/test3.txt", "Data/test4.txt", "Data/test5.txt"]
for i in range(4):
    I_range = np.array(range(1,11))*10**(-i-1)
    I_V_scan(list(I_range), namen[i])

I_range

array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009, 0.001 ])

In [23]:
I_range = np.linspace()

11